# __Flight Sentiment Analysis__

In [17]:
import pandas as pd
import numpy as np
import re
import nltk
import contractions
import warnings
import text_preprocessing as tp
from datetime import datetime
import utils

warnings.filterwarnings("ignore")

## Importing Dataset for Flight Metadata

Use the kaggle dataset containing information about flight reviews. Three different data sources will be utilized in the process of traning the model.

__a. Data Sources:__

https://data.world/akhilanil/airline-user-review/workspace/file?filename=airline.csv

https://www.kaggle.com/datasets/efehandanisman/skytrax-airline-reviews

https://www.kaggle.com/datasets/juhibhojani/airline-reviews

In [18]:
# Open file containing flight metadata
flight_metadata_df = pd.read_excel("./Dataset/Reviews/capstone_airline_reviews3.xlsx")
flight_metadata_df.head(5)

,airline,overall,author,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Turkish Airlines,7.0,Christopher Hackley,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,NaN,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Turkish Airlines,2.0,Adriana Pisoi,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,NaN,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# List all the columns in flight metadata's dataframe
flight_metadata_df.columns

Index(['airline', 'overall', 'author', 'review_date', 'customer_review',
       'aircraft', 'traveller_type', 'cabin', 'route', 'date_flown',
       'seat_comfort', 'cabin_service', 'food_bev', 'entertainment',
       'ground_service', 'value_for_money', 'recommended'],
      dtype='object')

In [20]:
# Drop columns that are not relevant
del_columns = ['author']
               
flight_metadata_df = flight_metadata_df.drop(del_columns, axis=1)

flight_metadata_df.head(5)

,airline,overall,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Turkish Airlines,7.0,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,NaN,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Turkish Airlines,2.0,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,NaN,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Rename the column names of flight dataset information
flight_metadata_df = flight_metadata_df.rename(columns={"overall" : "overall_rating"})
print(flight_metadata_df.columns)

Index(['airline', 'overall_rating', 'review_date', 'customer_review',
       'aircraft', 'traveller_type', 'cabin', 'route', 'date_flown',
       'seat_comfort', 'cabin_service', 'food_bev', 'entertainment',
       'ground_service', 'value_for_money', 'recommended'],
      dtype='object')


## Exploratory Data Analysis

Check and resolve duplicate records

In [22]:
# Drop duplicate records in flight metadata
print(f"Record count before removing duplicates => {flight_metadata_df.shape[0]}")
flight_metadata_df.drop_duplicates(inplace=True)
print(f"Record count after removing duplicates  => {flight_metadata_df.shape[0]}")

Record count before removing duplicates => 131895
Record count after removing duplicates  => 61184


Check and resolve missing values for `flight metadata`

In [23]:
# Check for missing values in flight metadata
flight_metadata_df.isna().sum()

airline                1
overall_rating      1783
review_date            1
customer_review        1
aircraft           42696
traveller_type     23644
cabin               2479
route              23671
date_flown         23750
seat_comfort        4973
cabin_service       4944
food_bev           12843
entertainment      20954
ground_service     24015
value_for_money     1857
recommended         1423
dtype: int64

In [24]:
# List of flight with missing information
flight_metadata_df[flight_metadata_df['customer_review'].isna()]

,airline,overall_rating,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# List of flight with missing information
flight_metadata_df[flight_metadata_df['airline'].isna()]

,airline,overall_rating,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# List of flight with missing information
flight_metadata_df[flight_metadata_df['aircraft'].isna()]

,airline,overall_rating,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Turkish Airlines,7.0,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,NaN,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
3,Turkish Airlines,2.0,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,NaN,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
5,Turkish Airlines,3.0,7th May 2019,âœ… Trip Verified | Rome to Prishtina via Ista...,NaN,Business,Economy Class,Rome to Prishtina via Istanbul,2019-05-01 00:00:00,1.0,4.0,1.0,3.0,1.0,2.0,no
9,Turkish Airlines,1.0,6th May 2019,âœ… Trip Verified | Mumbai to Dublin via Istan...,NaN,Solo Leisure,Economy Class,Mumbai to Dublin via Istanbul,2019-05-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131886,Ukraine International,NaN,21st January 2007,Two domestic flights between Kiev and Lviv las...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131888,Ukraine International,NaN,15th December 2006,I had a good flight from London to Kiev with U...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131890,Ukraine International,NaN,19th May 2006,Kiev - London (Gatwick) in business class (in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131892,Ukraine International,NaN,29th April 2006,Several flights - KBP to AMS (3 times one way)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no


In [27]:
# Replace all missing values inoverall_rating and customer_review with a blank
flight_metadata_df[['overall_rating', 'customer_review']] = flight_metadata_df[['overall_rating', 'customer_review']].fillna('')
flight_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61184 entries, 0 to 131894
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          61183 non-null  object 
 1   overall_rating   61184 non-null  object 
 2   review_date      61183 non-null  object 
 3   customer_review  61184 non-null  object 
 4   aircraft         18488 non-null  object 
 5   traveller_type   37540 non-null  object 
 6   cabin            58705 non-null  object 
 7   route            37513 non-null  object 
 8   date_flown       37434 non-null  object 
 9   seat_comfort     56211 non-null  float64
 10  cabin_service    56240 non-null  float64
 11  food_bev         48341 non-null  float64
 12  entertainment    40230 non-null  float64
 13  ground_service   37169 non-null  float64
 14  value_for_money  59327 non-null  float64
 15  recommended      59761 non-null  object 
dtypes: float64(6), object(10)
memory usage: 7.9+ MB


In [28]:
# Drop and analyse the number of records before and after dropping records with null values
print(f"Record count before removing records with null values => {flight_metadata_df.shape[0]}")
flight_metadata_df.dropna(inplace=True, axis=0)
print(f"Record count after removing records with null values  => {flight_metadata_df.shape[0]}")
flight_metadata_df.info()

Record count before removing records with null values => 61184
Record count after removing records with null values  => 13190
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13190 entries, 7 to 131712
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          13190 non-null  object 
 1   overall_rating   13190 non-null  object 
 2   review_date      13190 non-null  object 
 3   customer_review  13190 non-null  object 
 4   aircraft         13190 non-null  object 
 5   traveller_type   13190 non-null  object 
 6   cabin            13190 non-null  object 
 7   route            13190 non-null  object 
 8   date_flown       13190 non-null  object 
 9   seat_comfort     13190 non-null  float64
 10  cabin_service    13190 non-null  float64
 11  food_bev         13190 non-null  float64
 12  entertainment    13190 non-null  float64
 13  ground_service   13190 non-null  float64
 14  value_for_money  13190 

__Data Wrangling:__ 

Conduct data wrangling on `flight metadata` to transform and structure data into a desired format

In [29]:
# Define a function to extract the day, month, and year from date strings
def extract_date_info(date_str):
    # Use regular expressions to extract day, month, and year
    day_match = re.search(r'(\d+)(?:st|nd|rd|th)\s([a-zA-Z]+)\s(\d{4})', date_str)

    if day_match:
        day = int(day_match.group(1))
        month = day_match.group(2)
        year = int(day_match.group(3))

        # Convert month name to a numeric month
        month = datetime.strptime(month, '%B').month

        return day, month, year
    else:
        # Handle invalid date strings
        return None, None, None

In [30]:
# Apply the extract_date_info function to create new columns
flight_metadata_df['day'], flight_metadata_df['month'], flight_metadata_df['year'] = zip(*flight_metadata_df['review_date'].apply(extract_date_info))

In [31]:
# Display the converted day, month and year
selected_columns = ['review_date', 'day', 'month', 'year']
print(flight_metadata_df[selected_columns])

              review_date  day  month  year
7            6th May 2019    6      5  2019
15        29th April 2019   29      4  2019
17        29th April 2019   29      4  2019
19        28th April 2019   28      4  2019
29        24th April 2019   24      4  2019
...                   ...  ...    ...   ...
131676  10th October 2015   10     10  2015
131680   6th October 2015    6     10  2015
131696    6th August 2015    6      8  2015
131702     20th July 2015   20      7  2015
131712     17th June 2015   17      6  2015

[13190 rows x 4 columns]


Conduct data wrangling on `flight metadata` to transform and structure data into a desired format

__For Text Pre-processing:__ 
- Remove non-grammatical text like emails and URLs
- Replace non-ascii characters (some Python libraries are only limited to ascii characters like KeywordProcessor)
- Replace emojis with English word/s
- Handle contractions
- Handle slang words

In [32]:
emoji_dict = tp.get_emojis()
slang_word_dict = tp.webscrape_slang_words()

def initial_text_preprocessing(text):
    try:
        # Remove non-grammatical text
        text = tp.remove_email_address(text)
        text = tp.remove_hyperlink(text)

        # Replace non-ascii characters as there are Python libraries limiting this feature
        #text = tp.replace_nonascii_characters(text)

        # Replace emojis with English word/s
        text = emoji_dict.replace_keywords(text)

        # Handle contractions
        text = contractions.fix(text)

        # Replace slang words
        text = slang_word_dict.replace_keywords(text)

    except Exception as err:
        print(f"ERROR: {err}")
        print(f"Input Text: {text}")

    return text


flight_metadata_df[['customer_review']] = flight_metadata_df[['customer_review']].applymap(initial_text_preprocessing)
flight_metadata_df[['customer_review']]

,customer_review
7,âœ… Trip Verified | Flew on Turkish Airlines I...
15,âœ… Trip Verified | Basel to Cape Town via Ist...
17,Not Verified | Abu Dhabi to Luxembourg via Ist...
19,âœ… Trip Verified | The experience with Turkis...
29,âœ… Trip Verified | Houston to Kiev via Istanb...
...,...
131676,Ukraine International it is not for everyone. ...
131680,"Warsaw to Kiev the plane was on time, Kiev air..."
131696,Never flying them again and discourage anyone ...
131702,Clean and comfortable cabin. We were offered a...
